In [ ]:
import torch
import numpy as np
import onnx
import onnxruntime as rt
from typing import List, Tuple, Union, Dict

In [ ]:
EP_list = ['CUDAExecutionProvider', 'CPUExecutionProvider']

model_path = r"wd14_tagger_model\model.onnx"

# initialize the model.onnx
sess = rt.InferenceSession(model_path, providers=EP_list)

# get the outputs metadata as a list of :class:`onnxruntime.NodeArg`
output_name = sess.get_outputs()[0].name

# get the inputs metadata as a list of :class:`onnxruntime.NodeArg`
input_name = sess.get_inputs()[0].name

input = np.random.rand(1, 448, 448, 3).astype(np.float32)

# inference run using image_data as the input to the model 
detections = sess.run(None, {input_name:input })[0]


In [ ]:
model_path = r"wd14_tagger_model\model.onnx"
model = onnx.load(model_path)
for node in model.graph.node:
        print(node.name)

In [ ]:
# 添加更多层输出
for node in model.graph.node[-1:-5:-1]:
    for output in node.output:
        model.graph.output.extend([onnx.ValueInfoProto(name=output)])

In [ ]:
# 载入模型
providers = ['CUDAExecutionProvider']
ort_session = rt.InferenceSession(model.SerializeToString(), providers=providers)
outputs = [x.name for x in ort_session.get_outputs()]

In [ ]:
# 获取输出
input_arr = np.random.randint(0, 225, size=(1, 448, 448, 3)).astype(np.float32)
ort_inputs = {ort_session.get_inputs()[0].name: input_arr,}
ort_outs = ort_session.run(outputs, ort_inputs)

print(ort_outs)
print(ort_outs[0].shape)

In [ ]:
# 推理输入输出参数维度

value_info_protos = []
shape_info = onnx.shape_inference.infer_shapes(model)
for idx, node in enumerate(shape_info.graph.value_info):
    value_info_protos.append(node)

# 检查模型是否正确
onnx.checker.check_model(model)


In [ ]:
import torch
import numpy as np
a = torch.tensor( np.random.randint(0, 225, size=(500, 448, 448, 3)).astype(np.float32) ).to("cuda")
a.device

In [ ]:
def _gc():
    import torch
    import gc
    import time
    time.sleep(1)
    torch.cuda.empty_cache()
    gc.collect()

# del a

_gc()

In [ ]:
# 矩阵加维度

def reshape_array_to_2d(arr_list: List[np.ndarray]) -> List[np.ndarray]:
    """
    输入一个List[np.ndarry]，逐个检查其中元素，如果某个数据元素是一维的，就在其前面加一个维度，使其变成二维的
    注意，会直接修改传入的list中的数组本身
    """
    for i, arr in enumerate(arr_list):
        if isinstance(arr, np.ndarray) and arr.ndim == 1:
            arr.resize((1, arr.shape[0]))
    return arr_list


